**Домашнее задание**  
Потренеруем навыки, полученные на практическом занятии.  
Пользуясь новыми знаниями, проведите эксплоративный анализ данных на данных недвижимости Бостона, и попытайтесь обучить линейную модель наиболее качественно. Все ваши действия и решения описывайте в markdown.  
Данные вы сможете получить исполнив следующий код:  
from sklearn.datasets import load_boston  
import pandas as pd  
data = pd.read_csv(load_boston()[‘filename’], skiprows=1)  
Переменная, которую надо предсказать - ’MEDV’.  
Прочитать информацию о содержимом датасета:  
print(load_boston()[‘DESCR’])  
Если вы неудовлетворенны результатом, но лучшего решения предложить не можете, то высылайте задание на проверку.  
Цель задания - закрепить полученные навыки.

In [1]:
# :Attribute Information (in order):
#     - CRIM     per capita crime rate by town
#     - ZN       proportion of residential land zoned for lots over 25,000 sq.ft.
#     - INDUS    proportion of non-retail business acres per town
#     - CHAS     Charles River dummy variable (= 1 if tract bounds river; 0 otherwise)
#     - NOX      nitric oxides concentration (parts per 10 million)
#     - RM       average number of rooms per dwelling
#     - AGE      proportion of owner-occupied units built prior to 1940
#     - DIS      weighted distances to five Boston employment centres
#     - RAD      index of accessibility to radial highways
#     - TAX      full-value property-tax rate per $10,000
#     - PTRATIO  pupil-teacher ratio by town
#     - B        1000(Bk - 0.63)^2 where Bk is the proportion of blacks by town
#     - LSTAT    % lower status of the population
#     - MEDV     Median value of owner-occupied homes in $1000's

In [2]:
import pandas as pd

In [3]:
from sklearn.datasets import load_boston
data = pd.read_csv(load_boston()['filename'], skiprows=1)
data.head()

,CRIM,ZN,INDUS,CHAS,NOX,RM,AGE,DIS,RAD,TAX,PTRATIO,B,LSTAT,MEDV
0,0.00632,18.0,2.31,0,0.538,6.575,65.2,4.0900,1,296,15.3,396.90,4.98,24.0
1,0.02731,0.0,7.07,0,0.469,6.421,78.9,4.9671,2,242,17.8,396.90,9.14,21.6
2,0.02729,0.0,7.07,0,0.469,7.185,61.1,4.9671,2,242,17.8,392.83,4.03,34.7
3,0.03237,0.0,2.18,0,0.458,6.998,45.8,6.0622,3,222,18.7,394.63,2.94,33.4
4,0.06905,0.0,2.18,0,0.458,7.147,54.2,6.0622,3,222,18.7,396.90,5.33,36.2


Посмотрим, какие результаты дает первоначальный датасет.

In [4]:
# Обучаем модель линейной регрессии и смотрим на получившиеся веса, сдвиг и среднюю абсолютную ошибку

from sklearn.metrics import mean_absolute_error
from sklearn.linear_model import LinearRegression

# Вводим иксы
X = data[['CRIM', 'ZN', 'INDUS', 'CHAS', 'NOX', 'RM', 'AGE', 'DIS', 'RAD', 'TAX', 'PTRATIO', 'B', 'LSTAT']]
# Вводим игреки
y = data['MEDV']
# Обучаем модель
reg = LinearRegression().fit(X, y)
# Определяем веса (коэффициенты перед Х):
print('Weights: {}'.format(reg.coef_))
# Сдвиг
print('Bias: {}'.format(reg.intercept_))

# С помощью обученной модели и иксов предсказываем игреки
pred_values = reg.predict(data[['CRIM', 'ZN', 'INDUS', 'CHAS', 'NOX', 'RM', 'AGE', 'DIS', 'RAD', 'TAX', 'PTRATIO', 'B', 'LSTAT']])
# Считаем абсолютную ошибку
print('Error: {}'.format(mean_absolute_error(pred_values, y)))
print('Median:', y.median())
print(f'Error in %: {round((mean_absolute_error(pred_values, y) / y.median() * 100), 2)}')

Weights: [-1.08011358e-01  4.64204584e-02  2.05586264e-02  2.68673382e+00
 -1.77666112e+01  3.80986521e+00  6.92224640e-04 -1.47556685e+00
  3.06049479e-01 -1.23345939e-02 -9.52747232e-01  9.31168327e-03
 -5.24758378e-01]
Bias: 36.459488385090005
Error: 3.270862810900314
Median: 21.2
Error in %: 15.43


Используя логику и немного подглядывая в веса модели, откорректируем датасет и посмотрим, как изменится точность модели.

Показатель преступности оставляем. Он явно влияет на стоимость жилья.

Рассмотрим параметр доли территории, зонированной более 25000 кв. футов. С одной стороны, чем больше участок, тем более состоятельным должен быть покупатель, что отражается на стоимости и качестве здания, с другой - стоимость участков, разбитых на более мелкие части, в сумме дает большую цифру, чем неразбитый участок. Получается очень неоднозначно. Убираем.  
Посмотрим, как будет выглядеть модель без этого параметра.

In [5]:
# Обучаем модель линейной регрессии и смотрим на получившиеся веса, сдвиг и среднюю абсолютную ошибку

from sklearn.metrics import mean_absolute_error
from sklearn.linear_model import LinearRegression

# Вводим иксы
X = data[['CRIM', 'INDUS', 'CHAS', 'NOX', 'RM', 'AGE', 'DIS', 'RAD', 'TAX', 'PTRATIO', 'B', 'LSTAT']]
# Вводим игреки
y = data['MEDV']
# Обучаем модель
reg = LinearRegression().fit(X, y)
# Определяем веса (коэффициенты перед Х):
print('Weights: {}'.format(reg.coef_))
# Сдвиг
print('Bias: {}'.format(reg.intercept_))

# С помощью обученной модели и иксов предсказываем игреки
pred_values = reg.predict(data[['CRIM', 'INDUS', 'CHAS', 'NOX', 'RM', 'AGE', 'DIS', 'RAD', 'TAX', 'PTRATIO', 'B', 'LSTAT']])
# Считаем абсолютную ошибку
print('Error: {}'.format(mean_absolute_error(pred_values, y)))
print('Median:', y.median())
print(f'Error in %: {round((mean_absolute_error(pred_values, y) / y.median() * 100), 2)}')

Weights: [-9.83251164e-02 -1.55583378e-03  2.72974803e+00 -1.82288785e+01
  4.03004661e+00 -4.69768279e-03 -1.20397374e+00  2.82308788e-01
 -9.49628894e-03 -1.09041031e+00  9.41531798e-03 -5.15646448e-01]
Bias: 36.86325933562916
Error: 3.3307069805616605
Median: 21.2
Error in %: 15.71


Доля нерозничной коммерческой недвижимости прямо не должна влиять на стоимость жилых зданий, что подтверждает ее вес в модели.  
Посмотрим, как будет выглядеть модель без этого параметра.

In [6]:
# Обучаем модель линейной регрессии и смотрим на получившиеся веса, сдвиг и среднюю абсолютную ошибку

# Вводим иксы
X = data[['CRIM', 'ZN', 'CHAS', 'NOX', 'RM', 'AGE', 'DIS', 'RAD', 'TAX', 'PTRATIO', 'B', 'LSTAT']]
# Вводим игреки
y = data['MEDV']
# Обучаем модель
reg = LinearRegression().fit(X, y)
# Определяем веса (коэффициенты перед Х):
print('Weights: {}'.format(reg.coef_))
# Сдвиг
print('Bias: {}'.format(reg.intercept_))

# С помощью обученной модели и иксов предсказываем игреки
pred_values = reg.predict(data[['CRIM', 'ZN', 'CHAS', 'NOX', 'RM', 'AGE', 'DIS', 'RAD', 'TAX', 'PTRATIO', 'B', 'LSTAT']])
# Считаем абсолютную ошибку
print('Error: {}'.format(mean_absolute_error(pred_values, y)))
print('Median:', y.median())
print(f'Error in %: {round((mean_absolute_error(pred_values, y) / y.median() * 100), 2)}')

Weights: [-1.08419068e-01  4.59324221e-02  2.71640264e+00 -1.74295266e+01
  3.79702088e+00  6.97090941e-04 -1.48964290e+00  2.99874920e-01
 -1.17840125e-02 -9.47065386e-01  9.28181703e-03 -5.23466594e-01]
Bias: 36.36388200705616
Error: 3.2722501253652863
Median: 21.2
Error in %: 15.44


Участок будет явно дороже, если он граничит с рекой Чарльз, на которой проводится известная регата, а также расположены такие университеты как Harvard и MIT. Оставляем.

Концентрацию окиси азота, вероятно, невооруженным носом сложно измерить, и я бы исключил ее из датасета, но сравнительно высокой вес данного параметра не дает мне это сделать. Похоже, американцы используют дополнительные средства для выявления данного показателя, и для них он важен.

Среднее количество комнат в жилом помещении напрямую влияет на стоимость. Не в ста процентах случаев, но в основном так. Не трогаем.

Доля занятых собственниками зданий, построенных до 1940 г., на мой взгляд не имеет большого значения. Удаляем.  
Посмотрим, как будет выглядеть модель без этого параметра.

In [7]:
# Обучаем модель линейной регрессии и смотрим на получившиеся веса, сдвиг и среднюю абсолютную ошибку

# Вводим иксы
X = data[['CRIM', 'ZN', 'INDUS', 'CHAS', 'NOX', 'RM', 'DIS', 'RAD', 'TAX', 'PTRATIO', 'B', 'LSTAT']]
# Вводим игреки
y = data['MEDV']
# Обучаем модель
reg = LinearRegression().fit(X, y)
# Определяем веса (коэффициенты перед Х):
print('Weights: {}'.format(reg.coef_))
# Сдвиг
print('Bias: {}'.format(reg.intercept_))

# С помощью обученной модели и иксов предсказываем игреки
pred_values = reg.predict(data[['CRIM', 'ZN', 'INDUS', 'CHAS', 'NOX', 'RM', 'DIS', 'RAD', 'TAX', 'PTRATIO', 'B', 'LSTAT']])
# Считаем абсолютную ошибку
print('Error: {}'.format(mean_absolute_error(pred_values, y)))
print('Median:', y.median())
print(f'Error in %: {round((mean_absolute_error(pred_values, y) / y.median() * 100), 2)}')

Weights: [-1.08005604e-01  4.63336607e-02  2.05621774e-02  2.68902620e+00
 -1.77135399e+01  3.81439356e+00 -1.47861155e+00  3.05785940e-01
 -1.23286922e-02 -9.52211173e-01  9.32065314e-03 -5.23851840e-01]
Bias: 36.43692664760272
Error: 3.2706317845585002
Median: 21.2
Error in %: 15.43


Можно было бы удалить параметр удаленности от центров занятости, но по какой-то причине он имеет хороший отрицательный вес. Люди далеко на каждый день туда обращаются, а многие - никогда. Странно, но придется оставить.

Доступность радиальных магистралей сильно влият на стоимость.

Налоговая ставка также играет роль.

Чем меньше соотношение учеников к учителям, тем более интеллигентным получается усредненный взрослый человек, а именно по этому срезу населения и судят о районе. Вес в модели подтверждает эту догадку. Поэтому оставляем.

Убираем нетолерантное деление по цвету кожи. Вес в модели, к слову, очень низкий и даже положительный. Плюс, на глаз такое соотношение довольно сложно определить, если, конечно, речь не идет о нью-йоркском Гарлеме.  
Посмотрим, как будет выглядеть модель без этого параметра.

In [8]:
# Обучаем модель линейной регрессии и смотрим на получившиеся веса, сдвиг и среднюю абсолютную ошибку

# Вводим иксы
X = data[['CRIM', 'ZN', 'INDUS', 'CHAS', 'NOX', 'RM', 'AGE', 'DIS', 'RAD', 'TAX', 'PTRATIO', 'LSTAT']]
# Вводим игреки
y = data['MEDV']
# Обучаем модель
reg = LinearRegression().fit(X, y)
# Определяем веса (коэффициенты перед Х):
print('Weights: {}'.format(reg.coef_))
# Сдвиг
print('Bias: {}'.format(reg.intercept_))

# С помощью обученной модели и иксов предсказываем игреки
pred_values = reg.predict(data[['CRIM', 'ZN', 'INDUS', 'CHAS', 'NOX', 'RM', 'AGE', 'DIS', 'RAD', 'TAX', 'PTRATIO', 'LSTAT']])
# Считаем абсолютную ошибку
print('Error: {}'.format(mean_absolute_error(pred_values, y)))
print('Median:', y.median())
print(f'Error in %: {round((mean_absolute_error(pred_values, y) / y.median() * 100), 2)}')

Weights: [-1.21388618e-01  4.69634633e-02  1.34676947e-02  2.83999338e+00
 -1.87580220e+01  3.65811904e+00  3.61071055e-03 -1.49075365e+00
  2.89404521e-01 -1.26819813e-02 -9.37532900e-01 -5.52019101e-01]
Bias: 41.61727017595486
Error: 3.3563060455291502
Median: 21.2
Error in %: 15.83


Доля населения с низким статусом (взрослое население без шкльного образования и рабочие) очевидно негативно влияет на статус района, снижая стоимость жилья. Важный показатель. Оставляем.

Сведем все выводы (правильные и не очень) в общую модель и новый датасет.

In [9]:
data_corr = data.drop(['ZN', 'INDUS', 'AGE', 'B'], axis=1)
data_corr.head()

,CRIM,CHAS,NOX,RM,DIS,RAD,TAX,PTRATIO,LSTAT,MEDV
0,0.00632,0,0.538,6.575,4.0900,1,296,15.3,4.98,24.0
1,0.02731,0,0.469,6.421,4.9671,2,242,17.8,9.14,21.6
2,0.02729,0,0.469,7.185,4.9671,2,242,17.8,4.03,34.7
3,0.03237,0,0.458,6.998,6.0622,3,222,18.7,2.94,33.4
4,0.06905,0,0.458,7.147,6.0622,3,222,18.7,5.33,36.2


In [10]:
# Обучаем модель линейной регрессии и смотрим на получившиеся веса, сдвиг и среднюю абсолютную ошибку

# Вводим иксы
X = data_corr[['CRIM', 'CHAS', 'NOX', 'RM', 'DIS', 'RAD', 'TAX', 'PTRATIO', 'LSTAT']]
# Вводим игреки
y = data_corr['MEDV']
# Обучаем модель
reg = LinearRegression().fit(X, y)
# Определяем веса (коэффициенты перед Х):
print('Weights: {}'.format(reg.coef_))
# Сдвиг
print('Bias: {}'.format(reg.intercept_))

# С помощью обученной модели и иксов предсказываем игреки
pred_values = reg.predict(data_corr[['CRIM', 'CHAS', 'NOX', 'RM', 'DIS', 'RAD', 'TAX', 'PTRATIO', 'LSTAT']])
# Считаем абсолютную ошибку
print('Error: {}'.format(mean_absolute_error(pred_values, y)))
print('Median:', y.median())
print(f'Error in %: {round((mean_absolute_error(pred_values, y) / y.median() * 100), 2)}')

Weights: [-1.11462941e-01  2.86602898e+00 -1.95345227e+01  3.87503339e+00
 -1.19893871e+00  2.68457812e-01 -1.00484636e-02 -1.08200076e+00
 -5.46013436e-01]
Bias: 42.18608301936494
Error: 3.4165415513149737
Median: 21.2
Error in %: 16.12


Модель логически стала более правильной, но тест показывает, что ошибка в предсказании увеличилась на один процент.

Уважаемый преподаватель, очень жду ваш комментарий. В чем может быть загвоздка, и как с этим бороться?